# Code to Produce an Output from the Models
Presumably this will need to be transferred to a script-based format but this is the code to turn an input image into a controller action

## Loading Dependencies

In [ ]:
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Flatten, Input, Dense, Dropout, Lambda, Reshape, MaxPooling2D, LSTM, Reshape
import os
import cv2
import numpy as np


# Variables for potential modification

`img_height/width`: this and all below variables should be the same for the trained images and input images

`z_len`: the length of the image compression made by the encoder
`a_len`: the length of the action vector

In [ ]:
img_height = 128
img_width = 128
num_channels = 1
input_shape = (img_height, img_width, num_channels)
z_len = 2048
a_len = 1

## Model Architectures

# VAE

load the vae (have to make the architecture again, make sure the code below matches the code in the Data Prepper/VAE Trainer)


# Encoder
`latent_dim`: changing this will make the model exponentially larger or smaller

`x`: the model (saved in x)

`conv_shape`: Shape of conv to be provided to decoder

`z_mu` and `z_sigma`: Two outputs, for latent mean and log variance (std. dev.) Use these to sample random variables in latent space to which inputs are mapped. 

`sample_z` Function: REPARAMETERIZATION TRICK
- Define sampling function to sample from the distribution
- Reparameterize sample based on the process defined by Gunderson and Huang
- into the shape of: mu + sigma squared x eps
- This is to allow gradient descent to allow for gradient estimation accurately. 

In [ ]:
latent_dim = 2048

input_img = Input(shape=input_shape, name='encoder_input')
x = Conv2D(64, 3, padding='same', activation='relu')(input_img)
x = MaxPooling2D((2,2), padding='same')
x = Dropout(0.2)(x)
x = Conv2D(128, 3, padding='same', activation='relu')(x)
x = MaxPooling2D((2,2), padding='same')
x = Dropout(0.2)(x)
x = Conv2D(64, 3, padding='same', activation='relu')(x)
x = MaxPooling2D((2,2), padding='same')
x = Dropout(0.2)(x)
x = Conv2D(32, 3, padding='same', activation='relu')(x)

conv_shape = K.int_shape(x)
x = Flatten()(x)
x = Dense(latent_dim*2, activation='relu')(x)

z_mu = Dense(latent_dim, name='latent_mu')(x)
z_sigma = Dense(latent_dim, name='latent_sigma')(x)

def sample_z(args):
    z_mu, z_sigma = args
    eps = K.random_normal(shape=(K.shape(z_mu)[0], K.int_shape(z_mu)[1]))
    return z_mu + K.exp(z_sigma / 2) * eps

z = Lambda(sample_z, output_shape=(latent_dim,), name='z')([z_mu, z_sigma])

encoder = Model(input_img, [z_mu, z_sigma, z], name='encoder')

# Decoder 

### decoder takes the latent vector as input: `decoder_input`

First `x` Layer: Need to start with a shape that can be remapped to original image shape as we want our final utput to be same shape original input. So, add dense layer with dimensions that can be reshaped to desired output shape

Second `x` Layer: reshape to the shape of last conv. layer in the encoder, so we can 

Third `x` Layer: upscale (conv2D transpose) back to original shape use Conv2DTranspose to reverse the conv layers defined in the encoder

In [ ]:
decoder_input = Input()